### Importing Necessary Libraries

In [42]:
!pip install python-dotenv

In [44]:
import requests
import json
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os

In [45]:
load_dotenv()  # Load the .env file
api_key = os.getenv("WEATHERSTACK_API_KEY")
if not api_key:
    raise ValueError("API key not found. Check your .env file.")


# Fetching the weather data of Ikorodu, Lagos, using Latitude & Longititude from weatherstack
url = f"https://api.weatherstack.com/current?access_key={api_key}"

querystring = {"query":"6.510739384596194, 3.6162058548917946"}

response = requests.get(url, params=querystring)

print(response.json())

{'request': {'type': 'LatLon', 'query': 'Lat 6.51 and Lon 3.62', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Ikorodu', 'country': 'Nigeria', 'region': 'Lagos', 'lat': '6.617', 'lon': '3.517', 'timezone_id': 'Africa/Lagos', 'localtime': '2024-12-07 18:12', 'localtime_epoch': 1733595120, 'utc_offset': '1.0'}, 'current': {'observation_time': '05:12 PM', 'temperature': 31, 'weather_code': 113, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0001_sunny.png'], 'weather_descriptions': ['Sunny'], 'wind_speed': 17, 'wind_degree': 183, 'wind_dir': 'S', 'pressure': 1010, 'precip': 0, 'humidity': 59, 'cloudcover': 0, 'feelslike': 37, 'uv_index': 0, 'visibility': 8, 'is_day': 'yes'}}


In [4]:
# Write the json data into a json file
data = response.json()

filename = 'weather_report.json'

with open(filename, 'w') as file:
    json.dump(data, file, indent=4)

### Data Cleaning and Transformation

In [5]:
# Load JSON data from file
with open("weather_report.json", "r") as file:
    data = json.load(file)

# Preprocess arrays by flattening them into strings
def flatten_arrays(obj):
    if isinstance(obj, dict):
        return {k: flatten_arrays(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return ', '.join(map(str, obj))  # Convert lists to comma-separated strings
    return obj

# Flatten arrays in the JSON
data_flattened = flatten_arrays(data)

# Normalize the JSON data into a flat DataFrame
df = pd.json_normalize(data_flattened)

# Save to a CSV file
#df.to_csv("weather_data.csv", index=False)

In [61]:
df.head()

,request.type,request.query,request.language,request.unit,location.name,location.country,location.region,location.lat,location.lon,location.timezone_id,...,current.wind_degree,current.wind_dir,current.pressure,current.precip,current.humidity,current.cloudcover,current.feelslike,current.uv_index,current.visibility,current.is_day
0,LatLon,Lat 6.51 and Lon 3.62,en,m,Ikorodu,Nigeria,Lagos,6.617,3.517,Africa/Lagos,...,43,NE,1011,0,89,0,27,0,7,yes


In [62]:
df.columns

Index(['request.type', 'request.query', 'request.language', 'request.unit',
       'location.name', 'location.country', 'location.region', 'location.lat',
       'location.lon', 'location.timezone_id', 'location.localtime',
       'location.localtime_epoch', 'location.utc_offset',
       'current.observation_time', 'current.temperature',
       'current.weather_code', 'current.weather_icons',
       'current.weather_descriptions', 'current.wind_speed',
       'current.wind_degree', 'current.wind_dir', 'current.pressure',
       'current.precip', 'current.humidity', 'current.cloudcover',
       'current.feelslike', 'current.uv_index', 'current.visibility',
       'current.is_day'],
      dtype='object')

In [6]:
# Rename Columns
df.rename(columns={
    'location.name' : 'location_name',
    'location.country' : 'country',
    'location.region' : 'region',
    'location.lat' : 'latitude',
    'location.lon' : 'longititude',
    'location.timezone_id' : 'timezone_id',
    'location.localtime' : 'localtime',
    'location.localtime_epoch' : 'localtime_epoch',
    'location.utc_offset' : 'utc_offset',
    'current.observation_time' : 'observation_time',
    'current.temperature' : 'temperature',
    'current.weather_code' : 'weather_code',
    'current.weather_icons' : 'weather_icons',
    'current.weather_descriptions' : 'weather_descriptions',
    'current.wind_speed' : 'wind_speed',
    'current.wind_degree' : 'wind_degree',
    'current.wind_dir' : 'wind_dir',
    'current.pressure' : 'pressure',
    'current.precip' : 'precip',
    'current.humidity' : 'humidity',
    'current.cloudcover' : 'cloudcover',
    'current.feelslike' : 'feelslike',
    'current.uv_index' : 'uv_index',
    'current.visibility' : 'visibility',
    'current.is_day' : 'is_day'
}, inplace=True)

In [7]:
df.columns

Index(['request.type', 'request.query', 'request.language', 'request.unit',
       'location_name', 'country', 'region', 'latitude', 'longititude',
       'timezone_id', 'localtime', 'localtime_epoch', 'utc_offset',
       'observation_time', 'temperature', 'weather_code', 'weather_icons',
       'weather_descriptions', 'wind_speed', 'wind_degree', 'wind_dir',
       'pressure', 'precip', 'humidity', 'cloudcover', 'feelslike', 'uv_index',
       'visibility', 'is_day'],
      dtype='object')

In [8]:
#Convert LocalTime datatype to date
df['localtime'] = pd.to_datetime(df['localtime'])

In [9]:
# Take Necessary data
data_df = df[['location_name', 'country', 'region', 'latitude', 'longititude',
       'timezone_id', 'localtime', 'localtime_epoch', 'utc_offset',
       'observation_time', 'temperature', 'weather_code', 'weather_icons',
       'weather_descriptions', 'wind_speed', 'wind_degree', 'wind_dir',
       'pressure', 'precip', 'humidity', 'cloudcover', 'feelslike', 'uv_index',
       'visibility', 'is_day']]

In [85]:
#data_df.head()
data_df.columns

Index(['location_name', 'country', 'region', 'latitude', 'longititude',
       'timezone_id', 'localtime', 'localtime_epoch', 'utc_offset',
       'observation_time', 'temperature', 'weather_code', 'weather_icons',
       'weather_descriptions', 'wind_speed', 'wind_degree', 'wind_dir',
       'pressure', 'precip', 'humidity', 'cloudcover', 'feelslike', 'uv_index',
       'visibility', 'is_day'],
      dtype='object')

In [11]:
#Location Table
location = data_df[['location_name', 'country', 'region', 'latitude', 'longititude','timezone_id']].copy().reset_index(drop=True)
location['location_id'] = range(1, len(location) + 1)
location = location[['location_id','location_name', 'country', 'region', 'latitude', 'longititude',
       'timezone_id']]

C:\Users\Nonso\AppData\Local\Temp\ipykernel_14244\2884024703.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location['location_id'] = range(1, len(location) + 1)


In [13]:
#WeatherDescription Table
weatherDec = data_df[['weather_code', 'weather_icons', 'weather_descriptions']].copy().reset_index(drop=True)
weatherDec['weatherDec_id'] = range(1, len(weatherDec) + 1)
weatherDec = weatherDec[['weatherDec_id','weather_code', 'weather_icons', 'weather_descriptions']]

C:\Users\Nonso\AppData\Local\Temp\ipykernel_14244\1105441769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weatherDec['weatherDec_id'] = range(1, len(weatherDec) + 1)


In [18]:
# Weather fact table
weather_fact_table = data_df.merge(location, on=['location_name', 'country', 'region', 'latitude', 'longititude','timezone_id'], how='left') \
                            .merge(weatherDec, on=['weather_code', 'weather_icons', 'weather_descriptions'], how='left') \
                            [['location_id','weatherDec_id','localtime','localtime_epoch','utc_offset','observation_time','temperature','wind_speed','wind_degree','wind_dir','pressure', 'precip', 'humidity', 'cloudcover', 'feelslike', 'uv_index','visibility', 'is_day']]

weather_fact_table['Unique_id'] = range(1, len(weather_fact_table) + 1)

weather_fact_table = weather_fact_table[['Unique_id','location_id','weatherDec_id','localtime','localtime_epoch','utc_offset','observation_time','temperature','wind_speed','wind_degree','wind_dir','pressure', 'precip', 'humidity', 'cloudcover', 'feelslike', 'uv_index','visibility', 'is_day']]

In [19]:
weather_fact_table

,Unique_id,location_id,weatherDec_id,localtime,localtime_epoch,utc_offset,observation_time,temperature,wind_speed,wind_degree,wind_dir,pressure,precip,humidity,cloudcover,feelslike,uv_index,visibility,is_day
0,1,1,1,2024-12-07 15:33:00,1733585580,1.0,02:33 PM,35,16,175,S,1009,0,42,0,41,5,8,yes


In [20]:
# Loading to CSV
location.to_csv('location.csv')
weatherDec.to_csv('weatherDec.csv')
weather_fact_table.to_csv('weather_fact_table.csv')

### Data Storage (PostgreSQL)

In [22]:
# Develop a function to get the Database connection
def get_db_connection():
    connection = psycopg2.connect(
        host = 'localhost',
        database = 'sencrop',
        user = 'postgres',
        port = '5432',
        password = 'Nonsosky@1'
    )
    return connection

#connect to our database
conn = get_db_connection()

In [26]:
weather_fact_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Unique_id         1 non-null      int64         
 1   location_id       1 non-null      int64         
 2   weatherDec_id     1 non-null      int64         
 3   localtime         1 non-null      datetime64[ns]
 4   localtime_epoch   1 non-null      int64         
 5   utc_offset        1 non-null      object        
 6   observation_time  1 non-null      object        
 7   temperature       1 non-null      int64         
 8   wind_speed        1 non-null      int64         
 9   wind_degree       1 non-null      int64         
 10  wind_dir          1 non-null      object        
 11  pressure          1 non-null      int64         
 12  precip            1 non-null      int64         
 13  humidity          1 non-null      int64         
 14  cloudcover        1 non-null  

In [35]:
# Create a function that setups the schema and tables
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''
                            CREATE SCHEMA IF NOT EXISTS sencrop;

                            CREATE TABLE IF NOT EXISTS sencrop.location (
                                location_id SERIAL PRIMARY KEY,
                                location_name VARCHAR(100000),
                                country VARCHAR(100000),
                                region VARCHAR(100000),
                                latitude VARCHAR(100000),
                                longititude VARCHAR(100000),
                                timezone_id VARCHAR(10000)
                            );

                            CREATE TABLE IF NOT EXISTS sencrop.weatherDec (
                                weatherDec_id SERIAL PRIMARY KEY,
                                weather_code INTEGER,
                                weather_icons VARCHAR(100000),
                                weather_descriptions VARCHAR(100000)
                            );

                            CREATE TABLE IF NOT EXISTS sencrop.weather_fact_table (
                                Unique_id SERIAL PRIMARY KEY,
                                location_id INT,
                                weatherDec_id INT,
                                local_time TIMESTAMP,
                                localtime_epoch INT,
                                utc_offset VARCHAR(100000),
                                observation_time VARCHAR(100000),
                                temperature INT,
                                wind_speed INT,
                                wind_degree INT,
                                wind_dir VARCHAR(100000),
                                pressure INT,
                                precip INT,
                                humidity INT,
                                cloudcover INT,
                                feelslike INT,
                                uv_index INT,
                                visibility INT,
                                is_day VARCHAR(100000),
                                FOREIGN KEY (location_id) REFERENCES sencrop.location(location_id),
                                FOREIGN KEY (weatherDec_id) REFERENCES sencrop.weatherDec(weatherDec_id)
                            );
                            '''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

In [36]:
create_tables()

In [41]:
## Loading the data into the db tables
conn = get_db_connection()
cursor = conn.cursor()

cursor.execute('SELECT MAX(location_id) FROM sencrop.location')
last_id = cursor.fetchone()[0] or 0  # If table is empty, start from 0

# Iterate through rows and insert with incremented IDs
for i, (_, row) in enumerate(location.iterrows(), start=1):
    new_id = last_id + i
    cursor.execute(
        '''
        INSERT INTO sencrop.location(location_id, location_name, country, region, latitude, longititude, timezone_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ''',
        (new_id, row['location_name'], row['country'], row['region'], row['latitude'], row['longititude'], row['timezone_id'])
    )

for i, (_, row) in enumerate(weatherDec.iterrows(), start=1):
    new_id = last_id + i
    cursor.execute(
            '''INSERT INTO sencrop.weatherDec(weatherDec_id, weather_code, weather_icons,weather_descriptions)
            VALUES (%s, %s, %s, %s)''',
            (new_id, row['weather_code'], row['weather_icons'], row['weather_descriptions'])
)

for i, (_, row) in enumerate(weather_fact_table.iterrows(), start=1):
    new_id = last_id + i
    cursor.execute(
        '''INSERT INTO sencrop.weather_fact_table(Unique_id, location_id, weatherDec_id, local_time,localtime_epoch, utc_offset, observation_time, temperature, wind_speed, wind_degree, wind_dir, pressure, precip, humidity, cloudcover, feelslike, uv_index, visibility,is_day)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
            (new_id, row['location_id'], row['weatherDec_id'], row['localtime'], row['localtime_epoch'], row['utc_offset'], row['observation_time'], row['temperature'], row['wind_speed'], row['wind_degree'], row['wind_dir'], row['pressure'], row['precip'], row['humidity'], row['cloudcover'], row['feelslike'], row['uv_index'], row['visibility'], row['is_day'])
)
# Commit changes
conn.commit()

#close connection
cursor.close()
conn.close()